# Exploration of the UPenn PhotDec catalogs

The goal of this notebook is to play around with the data released by A. Meert and collaborators at  
http://alan-meert-website-aws.s3-website-us-east-1.amazonaws.com/fit_catalog/download/index.html

Relevant and related papers:
* [Vikram et al. 2010, *PyMorph: Automated Galaxy Structural Parameter Estimation using Python*](https://arxiv.org/abs/1007.4965)


* [Meert et al. 2013, *Simulations of single- and two-component galaxy decompositions for spectroscopically selected galaxies from the SDSS*](http://adsabs.harvard.edu/abs/2013MNRAS.433.1344M)
* [Meert et al. 2015, *A catalogue of 2D photometric decompositions in the SDSS-DR7 spectroscopic main galaxy sample: preferred models and systematics*](http://adsabs.harvard.edu/abs/2015MNRAS.446.3943M)
* [Meert et al. 2016, *A catalogue of 2D photometric decompositions in the SDSS-DR7 spectroscopic main galaxy sample: extension to g and i bands*](http://adsabs.harvard.edu/abs/2016MNRAS.455.2440M)  


* [Bernardi et al. 2013, *The massive end of the luminosity and stellar mass functions: dependence on the fit to the light profile*](http://adsabs.harvard.edu/abs/2013MNRAS.436..697B)
* [Bernardi et al. 2014, *Systematic effects on the size-luminosity relations of early- and late-type galaxies: dependence on model fitting and morphology*](http://adsabs.harvard.edu/abs/2014MNRAS.443..874B)
* [Bernardi et al. 2016, *The massive end of the luminosity and stellar mass functions and clustering from CMASS to SDSS: evidence for and against passive evolution*](http://adsabs.harvard.edu/abs/2016MNRAS.455.4122B)
* [Bernardi et al. 2017a, *The high mass end of the stellar mass function: Dependence on stellar population models and agreement between fits to the light profile*](http://adsabs.harvard.edu/abs/2017MNRAS.467.2217B)


* [Fischer et al. 2017, *Comparing pymorph and SDSS photometry - I. Background sky and model fitting effects*](http://adsabs.harvard.edu/abs/2017MNRAS.467..490F)
* [Bernardi et al. 2017b, *Comparing pymorph and SDSS photometry - II. The differences are more than semantics and are not dominated by intracluster light*](http://adsabs.harvard.edu/abs/2017MNRAS.468.2569B)


* [Mendel et al. 2014, *A Catalog of Bulge, Disk, and Total Stellar Mass Estimates for the Sloan Digital Sky Survey*](http://adsabs.harvard.edu/abs/2014ApJS..210....3M)

### Imports and paths

In [1]:
import os
import numpy as np

import fitsio
import matplotlib.pyplot as plt
from corner import corner

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
basicdir = os.path.join(os.getenv('IM_DATA_DIR'), 'upenn-photdec', 'basic-catalog', 'v2')
adddir = os.path.join(os.getenv('IM_DATA_DIR'), 'upenn-photdec', 'additional-catalogs')

### Read the parent CAST catalog.
Read the parent SDSS (*CAST*) catalog which defines the sample.

In [4]:
castfile = os.path.join(basicdir, 'UPenn_PhotDec_CAST.fits')
castinfo = fitsio.FITS(castfile)
castinfo[1]


  file: /Users/ioannis/research/data/upenn-photdec/basic-catalog/v2/UPenn_PhotDec_CAST.fits
  extension: 1
  type: BINARY_TBL
  rows: 670722
  column info:
    galcount            i4  
    objid               i8  
    SDSSIAU            S24  
    badflag             i4  
    nchild              i2  
    mode                i2  
    run                 i2  
    rerun               i2  
    camCol              i2  
    field               i2  
    obj                 i2  
    stripe              i4  
    startmu             i4  
    specobjid           i8  
    plate               i2  
    mjd                 i4  
    fiberid             i2  
    ra                  f4  
    dec                 f4  
    z                   f4  
    veldisp             f4  
    veldispErr          f4  
    eclass              f4  
    p_el_debiased       f4  
    p_cs_debiased       f4  
    spiral              i4  
    elliptical          i4  
    uncertain           i4  

In [6]:
allcast = castinfo[1].read()

### Read the grz model fitting results and select a "good" sample.
Read the model fitting results (in each band) and select a clean sample using the "finalflag" bit (see Section 2.2 of the `data_tables.pdf` documentation).

In [8]:
band = ['gband', 'rband', 'iband']

In [10]:
def select_good(finalflag, n_bulge):
    """Select a good sample of targets."""
    noflags = finalflag & 11 != 0
    goodserexp = finalflag & 12 != 0
    flip = finalflag & 13 != 0
    smalln = n_bulge < 8

    good = np.logical_or(noflags, goodserexp)
    good = np.logical_or(good, flip)
    good = np.logical_and(good, smalln)

    isbulge = finalflag & 1 != 0
    isser = finalflag & 4 != 0
    isfit = np.logical_or(isbulge, isser)
    
    good = np.logical_and(good, isfit)
    
    return good

In [33]:
band = 'rband'
ff = fitsio.read(os.path.join(basicdir, 'UPenn_PhotDec_Models_rband.fits'), ext=1)
ff.dtype.names

('m_tot',
 'm_aper',
 'BT',
 'r_tot',
 'ba_tot',
 'BT_aper',
 'xctr_bulge',
 'xctr_bulge_err',
 'yctr_bulge',
 'yctr_bulge_err',
 'm_bulge',
 'm_bulge_err',
 'r_bulge',
 'r_bulge_err',
 'n_bulge',
 'n_bulge_err',
 'ba_bulge',
 'ba_bulge_err',
 'pa_bulge',
 'pa_bulge_err',
 'xctr_disk',
 'xctr_disk_err',
 'yctr_disk',
 'yctr_disk_err',
 'm_disk',
 'm_disk_err',
 'r_disk',
 'r_disk_err',
 'n_disk',
 'n_disk_err',
 'ba_disk',
 'ba_disk_err',
 'pa_disk',
 'pa_disk_err',
 'GalSky',
 'GalSky_err',
 'chi2nu',
 'finalflag',
 'autoflag',
 'pyflag',
 'pyfitflag')

In [52]:
print(ff['finalflag'][1])
print( np.sum(ff['finalflag'] & 20 != 0) )
print( np.sum( (ff['finalflag'] & 20 != 0) * (ff['finalflag'] & 10 != 0) ) )

81
292825
127574


In [43]:
for bb in np.arange(27):
    print(bb, ff['finalflag'][1] & bb)

0 0
1 1
2 0
3 1
4 0
5 1
6 0
7 1
8 0
9 1
10 0
11 1
12 0
13 1
14 0
15 1
16 16
17 17
18 16
19 17
20 16
21 17
22 16
23 17
24 16
25 17
26 16


In [11]:
good = None
for thisband in band:
    measfile = os.path.join(basicdir, 'UPenn_PhotDec_NonParam_{}.fits'.format(thisband))
    measinfo = fitsio.FITS(measfile)
    fitfile = os.path.join(basicdir, 'UPenn_PhotDec_Models_{}.fits'.format(thisband))
    fitinfo = fitsio.FITS(fitfile)    
    _fit = fitinfo[1].read(columns=['finalflag', 'n_bulge'])
    _good = select_meert(_fit['finalflag'], _fit['n_bulge'])
    if good is None:
        print(measinfo[1])
        print(fitinfo[1])
        good = _good        
    else:
        good = np.logical_and(good, _good)
goodindx = np.where(good)[0]
nobj = len(goodindx)
print('Selected {}/{} good targets.'.format(nobj, len(_fit)))


  file: /Users/ioannis/research/data/upenn-photdec/basic-catalog/v2/UPenn_PhotDec_NonParam_gband.fits
  extension: 1
  type: BINARY_TBL
  rows: 670722
  column info:
    SexMag              f4  
    SexMag_Err          f4  
    SexHrad             f4  
    SexSky              f4  
    num_targets         i2  
    num_neighborfit     i2  
    AperRad             f4  
    C                   f4  
    C_err               f4  
    A                   f4  
    A_err               f4  
    S                   f4  
    S_err               f4  
    G                   f4  
    M20                 f4  
    extinction          f4  
    dismod              f4  
    kpc_per_arcsec      f4  
    Vmax                f4  
    SN                  f4  
    kcorr               f4  

  file: /Users/ioannis/research/data/upenn-photdec/basic-catalog/v2/UPenn_PhotDec_Models_gband.fits
  extension: 1
  type: BINARY_TBL
  rows: 670722
  column info:
    m_tot               f4  
    m_aper              f4  
 

In [ ]:
fit, meas = [], []
for thisband in band:
    fitfile = os.path.join(basicdir, 'UPenn_PhotDec_Models_{}.fits'.format(thisband))
    measfile = os.path.join(basicdir, 'UPenn_PhotDec_NonParam_{}.fits'.format(thisband))
    fit.append(fitsio.read(fitfile, ext=1, rows=goodindx))
    meas.append(fitsio.read(measfile, ext=1, rows=goodindx))
gfit, rfit, ifit = fit
gmeas, rmeas, imeas = meas
del fit, meas

### Read supplemental catalogs.

Read additional, supplemental catalogs which we will analyze below: the *CASTModel* catalog (SDSS photometry) and the stellar masses catalog based on the dust-free mass-to-light ratios from Mendel et al. (2014) (their Table 5).

In [ ]:
castmodfile = os.path.join(adddir, 'UPenn_PhotDec_CASTmodels.fits')
castmodinfo = fitsio.FITS(castmodfile)
castmodinfo[1]

In [ ]:
castmod = castmodinfo[1].read(rows=goodindx)

In [ ]:
mendelfile = os.path.join(adddir, 'UPenn_PhotDec_Mstar_mlMendel14.dat')
columns = ('GalCount', 'FlagSerExp', 'Mstar_Tab5_Pymorph',
           'Mstar_Tab5_Truncated', 'Mstar_Tab3_Pymorph',
           'Mstar_Tab3_Truncated', 'Mstar_Tab5_Mendel',
           'Mstar_Tab3_Mendel', 'Mstar_Tab5_cModel',
           'Mstar_Tab3_cModel')
dtype = np.dtype([(col, np.float) for col in columns])
allmendel = np.loadtxt(mendelfile, dtype=dtype)


In [ ]:
allmendel

### Generate various diagnostic plots

In [ ]:
print('r-band range = {:.3f} - {:.3f}'.format(rfit['m_tot'].min(), rfit['m_tot'].max()))
print('Redshift range = {:.4f} - {:.4f}'.format(cast['z'].min(), cast['z'].max()))

In [ ]:
fig, ax = plt.subplots()
cm = plt.cm.get_cmap('RdYlBu')
hb = ax.hexbin(cast['ra'], cast['dec'], C=cast['z'], cmap=cm, 
               vmin=0, vmax=0.3)
cb = plt.colorbar(hb)
cb.set_label('Redshift')

In [ ]:
fig, ax = plt.subplots()
_, _, _ = ax.hist(cast['z'], bins=100, range=(0, 0.4), alpha=0.5)
ax.set_xlabel('Redshift')
ax.set_ylabel('Number of Galaxies')

In [ ]:
fig, ax = plt.subplots()
ax.scatter(cast['z'], rfit['m_tot'], s=1)
ax.set_xlim((0, 0.4))
ax.margins(0.05)

In [ ]:
labels = ['r (AB mag)', 'g - r', 'Sersic n', 'Half-light Radius (arcsec)']
data = np.array([ rfit['m_tot'], gfit['m_tot'] - rfit['m_tot'], rfit['n_disk'], rfit['r_disk'] ]).T
data.shape

In [ ]:
_ = corner(data, quantiles=[0.25, 0.50, 0.75], labels=labels)